# COGS 108 - Data Checkpoint

# Names

- Asher Av
- Quoc-Zuy  Do
- Hector Gallo
- Jeremy Nurding
- Andres Villegas

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.


**Data Set Name: COVID-19 Data - US Counties from NYTimes**
- Link to Dataset: https://github.com/nytimes/covid-19-data
- Number of Obsevations: 2,170,941 
- <ins>Description of Dataset:</ins> This dataset is collated from  data across the U.S. by the New York Times and draws from the official reportings about the cumulative number of cases and deaths reported in each county and state across the U.S since the start of the COVID-19 pandemic. This dataset contains 6 columns of  data: date, county, state, fips, cases and  deaths. The FIPS column  crefers to a FIPS code, a geographic identifier that determines the location of the county the data was pulled from and makes it easy to associate with other datasets

**Data Set Name: Air Quality Across Countries in COVID-19**
- Link to Dataset: https://www.kaggle.com/aestheteaman01/air-quality-across-countries-in-covid19/version/3?select=USA.csv
- Number of Observations: 179,365
- <ins>Description of Dataset:</ins> This dataset shows the collected information of the air quality from 2020-2021 across countries including Brazil, Canada, France, Italy, India, and USA and their respective cities. The parameters that they use to show the air quality for each of the cities are the following: Carbon Monoxide (CO), Dew (dew), Humidity (humidity), Nitrogen Dioxide (NO2), Ozone (O3), Particulate Matter 10 (pm10, Particulate Matter 2.5 (pm25), Pressure (pressure), Sulphur Dioxide (SO2), Temperature, Wind Gusts, and Wind Speed.

**Data Set Name: Air Quality Statistics by County, 2020**
- Link to Dataset: https://www.epa.gov/air-trends/air-quality-cities-and-counties 
- Number of Observations: 1,144
- <ins>Description of Dataset:</ins> The data set “Air Quality Statistics by County, 2020” presents data of counties with highest levels of air pollutants across states in the United States.  This data set is organized alphabetically in rows by state, each county is assigned to its state accordingly and also arranged in alphabetical order.  There are a total of 1162 rows in this data set and out of the total of rows however 1144 rows contain observations of air pollutants for specific counties.  This dataset contains 13 columns from left to right as follows: State, County, County FIPS Code, 2010 population, CO, Pb, NO2 AM (ppb), NO2 hr (ppb), O3, PM10, PM2.5 Wtd AM, PM2.5 hr, and SO2.  These show the highest number of such pollutant in the given area.

# Setup

##### Import Modules

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# nytimes daily covid dataset
covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

# Kaggle air quality dataset
kagaq_df = pd.read_csv('https://raw.githubusercontent.com/asherbav/covid_pollution_files/main/USA.csv')

# EPA air quality dataset 
epa_df = pd.read_csv('https://raw.githubusercontent.com/asherbav/covid_pollution_files/main/ctyfactbook2020.csv')


# Data Cleaning

Describe your data cleaning steps here.

In [3]:
covid_df.shape

(2193709, 6)

In [4]:
covid_df.isnull().sum()

date          0
county        0
state         0
fips      20364
cases         0
deaths    50195
dtype: int64

In [5]:
covid_df = covid_df.drop(['deaths'], axis = 1)

In [6]:
covid_df = covid_df.dropna()

In [7]:
covid_df.shape

(2173345, 5)

In [9]:
covid_df.isnull().sum()

date      0
county    0
state     0
fips      0
cases     0
dtype: int64

In [8]:
kagaq_df.isnull().sum()

Date        0
Country     0
City        0
Specie      0
count       0
min         0
max         0
median      0
variance    0
dtype: int64